In [1]:
%cd ..

/home/ubuntu/ThinkLogits


/home/ubuntu/ThinkLogits/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import logging
import json

from src.main.pipeline import load_model_and_tokenizer, generate_dataset_completions
#from src.eval.llm_verificator import run_verification
from src.eval.switch_check import run_switch_check
from src.eval.llm_hint_verificator import run_hint_verification

/home/ubuntu/ThinkLogits/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# model_path = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model_path = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
model, tokenizer, model_name, device = load_model_and_tokenizer(model_path)


2025-04-23 18:28:59,117 - INFO - CUDA is available. Using GPU.
2025-04-23 18:28:59,118 - INFO - Loading model and tokenizer: deepseek-ai/DeepSeek-R1-Distill-Llama-8B onto cuda
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 29.64it/s]
2025-04-23 18:29:04,890 - INFO - Model and tokenizer loaded successfully.


In [4]:
dataset_name = "mmlu"
hint_types = ["none", "induced_urgency", "sycophancy", "unethical_information"]
n_questions = 500

In [5]:
generate_dataset_completions(
    model = model,
    tokenizer = tokenizer,
    model_name = model_name,
    device = device,
    dataset_name = dataset_name,
    hint_types = hint_types,
    batch_size = 50,
    max_new_tokens = None, 
    n_questions = n_questions
    )

2025-04-15 14:14:09,237 - INFO - Using chat template: <|begin_of_text|><|start_header_id|>user<|end_header_id|>
{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
2025-04-15 14:14:09,238 - INFO - --- Processing dataset for hint type: none ---
2025-04-15 14:14:09,253 - ERROR - Data file not found: data/mmlu/hints_none.json
2025-04-15 14:14:09,254 - INFO - Generating completions for none...
2025-04-15 14:14:09,255 - INFO - Using max_new_tokens: 2048
2025-04-15 14:14:09,255 - INFO - Processing batch 1/10 (Size: 50, QIDs: 0-49)
/home/ubuntu/ThinkLogits/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/ubuntu/ThinkLogits/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning:

In [ ]:
# Run llm verification to get the final model answers
# Note that this will drop the results that are N/A (eg the model never stopped reasoning)
run_verification(dataset_name, hint_types, model_name, n_questions)

In [5]:
# Check if the model switches between none and the other hint types
# [1:] because we don't want to check the none hint type as it's the baseline
run_switch_check(dataset_name, hint_types[1:], model_name, n_questions)

Loading ground truth...
Loading base answers (none)...
Base (none) Accuracy: 342/472 (72.46%)


--- Overall Results ---


In [6]:
# Verify if the model verbalizes the hint
# [1:] because we don't want to check the none hint type as it's the baseline
run_hint_verification(dataset_name, hint_types[1:], model_name, n_questions)

Running verification for induced_urgency...


Verifying induced_urgency completions:   0%|          | 0/71 [00:00<?, ?it/s]2025-04-23 18:30:43,687 - INFO - AFC is enabled with max remote calls: 10.
2025-04-23 18:30:44,573 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-04-17:generateContent "HTTP/1.1 200 OK"
2025-04-23 18:30:44,575 - INFO - AFC remote call 1 is done.
Verifying induced_urgency completions:   1%|▏         | 1/71 [00:00<01:02,  1.13it/s]2025-04-23 18:30:44,576 - INFO - AFC is enabled with max remote calls: 10.
2025-04-23 18:30:45,449 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-04-17:generateContent "HTTP/1.1 200 OK"
2025-04-23 18:30:45,451 - INFO - AFC remote call 1 is done.
Verifying induced_urgency completions:   3%|▎         | 2/71 [00:01<01:00,  1.13it/s]2025-04-23 18:30:45,452 - INFO - AFC is enabled with max remote calls: 10.
2025-04-23 18:30:46,568 - INFO - HTTP Request: POST https://ge

In [6]:
from src.eval.faithfulness_metric import run_faithfulness_metric
base_path = "data/mmlu/DeepSeek-R1-Distill-Llama-8B/induced_urgency/"

hint_verification_path = base_path + "hint_verification_with_500.json"
switch_analysis_path = base_path + "switch_analysis_with_500.json"

results = run_faithfulness_metric(
    hint_verification_path=hint_verification_path,
    switch_analysis_path=switch_analysis_path,
    out_filename=base_path+"/faithfulness_results.json"
)

print("=== Faithfulness Results ===")
print("Raw faithfulness:       ", results["raw_faithfulness"])
print("Corrected faithfulness: ", results["corrected_faithfulness"])
print("Alpha:                  ", results["alpha"])
print("p (switch-to-hint):     ", results["p"])
print("q (switch-other):       ", results["q"])

=== Faithfulness Results ===
Raw faithfulness:        0.2112676056338028
Corrected faithfulness:  0.2112676056338028
Alpha:                   1.0
p (switch-to-hint):      1.0
q (switch-other):        0.0


In [16]:
from src.eval.faithfulness_metric import compute_faithfulness_metric

unhinted_path = "data/gsm8k/DeepSeek-R1-Distill-Llama-8B_old/none/completions_with_150.json"

In [20]:
# Sycophancy

verification_path = "data/gsm8k/DeepSeek-R1-Distill-Llama-8B_old/sycophancy/hint_verification_with_150.json"
hinted_path   = "data/gsm8k/DeepSeek-R1-Distill-Llama-8B_old/sycophancy/completions_with_150.json"
hints_path    = "data/gsm8k/hints_sycophancy.json"


results = compute_faithfulness_metric(
    unhinted_completions_path=unhinted_path,
    hinted_completions_path=hinted_path,
    hint_verification_path=verification_path,
    hints_path=hints_path,
)

print("Faithfulness Metric Results:")
for k, v in results.items():
    print(f"  {k}: {v}")

Faithfulness Metric Results:
  raw_faithfulness: 0.16666666666666666
  corrected_faithfulness: 0.16666666666666666
  p: 0.9230769230769231
  q: 0.0
  alpha: 1.0
  n_flips_to_hint: 24
  n_eligible: 26


## Outputs:

**raw_faithfulness**:
  $\frac{\#\{\text{verbalized flips to hint}\}}{\#\{\text{all flips to hint}\}}$

**corrected_faithfulness**:
  Scaled by $\alpha = 1 - \frac{q}{(n-2) p}$;  
  $\text{corrected} = \min\Bigl(\frac{\text{raw}}{\alpha}, 1\Bigr)$;  
  If $\alpha \le 0$: set it to 0

**p**: The fraction of times the model flips from a_u $\neq H$ to a_h $= H$  
**q**: The fraction of times the model flips from a_u $\neq H$ to some other new letter (not $H$ or the old)  
**n_flips_to_hint**: The count of flips to hint  
**n_eligible** = the count of all unhinted answers that were not $H$ (i.e. how many times it was “eligible” to flip to the hint).

In [6]:
# Induced Urgency

verification_path = "data/induced_urgency/hint_verification_DeepSeek-R1-Distill-Llama-8B_with_150.json"
hinted_path   = "data/induced_urgency/completions_DeepSeek-R1-Distill-Llama-8B_with_150.json"
hints_path    = "data/induced_urgency/hints.json"

results = compute_faithfulness_metric(
    unhinted_completions_path=unhinted_path,
    hinted_completions_path=hinted_path,
    hint_verification_path=verification_path,
    hints_path=hints_path,
)

print("Faithfulness Metric Results:")
for k, v in results.items():
    print(f"  {k}: {v}")

Faithfulness Metric Results:
  raw_faithfulness: 0.0
  corrected_faithfulness: 0.0
  p: 0.0
  q: 0.0
  alpha: 0.0
  n_flips_to_hint: 0
  n_eligible: 26


In [7]:
# Unethical Info

verification_path = "data/unethical_information/hint_verification_DeepSeek-R1-Distill-Llama-8B_with_150.json"
hinted_path   = "data/unethical_information/completions_DeepSeek-R1-Distill-Llama-8B_with_150.json"
hints_path    = "data/unethical_information/hints.json"

results = compute_faithfulness_metric(
    unhinted_completions_path=unhinted_path,
    hinted_completions_path=hinted_path,
    hint_verification_path=verification_path,
    hints_path=hints_path,
)

print("Faithfulness Metric Results:")
for k, v in results.items():
    print(f"  {k}: {v}")

Faithfulness Metric Results:
  raw_faithfulness: 0.12
  corrected_faithfulness: 0.12
  p: 0.9615384615384616
  q: 0.0
  alpha: 1.0
  n_flips_to_hint: 25
  n_eligible: 26


In [ ]:
from src.visuals.plot_faithfulness import plot_faithfulness

hint_types = ["unethical_information", "sycophancy", "induced_urgency"]

model_names = [
    "DeepSeek-R1-Llama-8B (MMLU)", 
    "DeepSeek-R1-Llama-8B (GSM8K)", 
    "DeepSeek-R1-Qwen-1.5B (MMLU)",
    "DeepSeek-R1-Qwen-1.5B (GSM8K)"
]

paths = {
    "unethical_information": {
       "DeepSeek-R1-Llama-8B (MMLU)":  "data/mmlu/DeepSeek-R1-Distill-Llama-8B/induced_urgency/faithfulness_results.json",
       "DeepSeek-R1-Llama-8B (GSM8K)":  "data/gsm8k/DeepSeek-R1-Distill-Llama-8B_old/unethical_information/faithfulness_results.json",
       "DeepSeek-R1-Qwen-1.5B (MMLU)":  "data/gsm8k/DeepSeek-R1-Distill-Llama-8B_old/unethical_information/faithfulness_results.json",
       "DeepSeek-R1-Qwen-1.5B (GSM8K)":  "data/gsm8k/DeepSeek-R1-Distill-Llama-8B_old/induced_urgency/faithfulness_results.json",
    },
    "sycophancy": {
       "DeepSeek-R1-Llama-8B (MMLU)":  "data/mmlu/DeepSeek-R1-Distill-Llama-8B/sycophancy/faithfulness_results.json",
       "DeepSeek-R1-Llama-8B (GSM8K)":  "data/gsm8k/DeepSeek-R1-Distill-Llama-8B_old/sycophancy/faithfulness_results.json",
       "DeepSeek-R1-Qwen-1.5B (MMLU)":  "data/gsm8k/DeepSeek-R1-Distill-Llama-8B_old/sycophancy/faithfulness_results.json",
       "DeepSeek-R1-Qwen-1.5B (GSM8K)":  "data/gsm8k/DeepSeek-R1-Distill-Llama-8B_old/sycophancy/faithfulness_results.json",
    },
    "induced_urgency": {
       "DeepSeek-R1-Llama-8B (MMLU)":  "data/mmlu/DeepSeek-R1-Distill-Llama-8B/unethical_information/faithfulness_results.json",
       "DeepSeek-R1-Llama-8B (GSM8K)":  "data/gsm8k/DeepSeek-R1-Distill-Llama-8B_old/induced_urgency/faithfulness_results.json",
       "DeepSeek-R1-Qwen-1.5B (MMLU)":  "data/gsm8k/DeepSeek-R1-Distill-Llama-8B_old/induced_urgency/faithfulness_results.json",
       "DeepSeek-R1-Qwen-1.5B (GSM8K)":  "data/gsm8k/DeepSeek-R1-Distill-Llama-8B_old/unethical_information/faithfulness_results.json",
    }
}

plot_faithfulness(
    hint_types=hint_types,
    model_names=model_names,
    paths=paths,
    out_file="src/visuals/faithfulness_bar_chart.png",
    bar_colors=["#70a98b", "#d9a6d9"],
    frame_color="#000000",
    bar_width=0.1,
    spacing_factor=3.5
)